In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import explode
from pyspark.sql.functions import split

spark = SparkSession \
    .builder \
    .appName("StructuredNetworkWordCount") \
    .getOrCreate()

In [2]:
lines = spark \
    .readStream \
    .format("socket") \
    .option("host", "localhost") \
    .option("port", 9999) \
    .load()

In [ ]:
words = lines.select(
   explode(
       split(lines.value, " ")
   ).alias("word")
)

In [11]:
wordCounts = words.groupBy("word").count()

In [12]:
query = wordCounts \
    .writeStream \
    .outputMode("complete") \
    .format("console") \
    .start()


In [13]:
query.awaitTermination()

StreamingQueryException: 'Connection timed out (Connection timed out)\n=== Streaming Query ===\nIdentifier: [id = 5fbd60b9-5274-428f-9542-3c09edf99bde, runId = a61e5a5c-00d9-4c84-8805-4e75d938a228]\nCurrent Committed Offsets: {}\nCurrent Available Offsets: {TextSocketV2[host: localhost, port: 9999]: -1}\n\nCurrent State: ACTIVE\nThread State: RUNNABLE\n\nLogical Plan:\nAggregate [word#86], [word#86, count(1) AS count#90L]\n+- Project [word#86]\n   +- Generate explode(split(value#83,  )), false, [word#86]\n      +- StreamingExecutionRelation TextSocketV2[host: localhost, port: 9999], [value#83]\n'